# 도서 추천 Toy 프로젝트
Kaggle의 도서 데이터를 바탕으로 유저에게 새로운 도서를 추천하는 추천 시스템을 만들어보고자 한다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
books = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BookRecommendation/Books.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BookRecommendation/Ratings.csv')
users = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BookRecommendation/Users.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
users

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


## Data_처리

In [5]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [6]:
# isbn, 책 제목, 저자, 년도, 출판사 정보만 남긴다.
books = books.iloc[:,0:4]

In [7]:
# 컬럼명을 편하게 변경
books.columns = ['ISBN', 'title', 'author', 'pub_year']

In [8]:
books.isna().sum()

ISBN        0
title       0
author      1
pub_year    0
dtype: int64

In [9]:
users

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [10]:
# user 나이에 Nan이 많이 보인다. 연령대별로 어떤 책을 좋아하는지도 볼 수 있으니 나이를 쓸 수 있을지를 확인해보자.
users.isna().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [11]:
# location의 경우 마지막의 국가만 활용해보려고 한다.
def cut_loc(loc):
  loc_cut_list = list(loc.split(', '))
  return loc_cut_list[-1]

In [12]:
users['country'] = users['Location'].apply(lambda x : cut_loc(x))

In [13]:
users

,User-ID,Location,Age,country
0,1,"nyc, new york, usa",NaN,usa
1,2,"stockton, california, usa",18.0,usa
2,3,"moscow, yukon territory, russia",NaN,russia
3,4,"porto, v.n.gaia, portugal",17.0,portugal
4,5,"farnborough, hants, united kingdom",NaN,united kingdom
...,...,...,...,...
278853,278854,"portland, oregon, usa",NaN,usa
278854,278855,"tacoma, washington, united kingdom",50.0,united kingdom
278855,278856,"brampton, ontario, canada",NaN,canada
278856,278857,"knoxville, tennessee, usa",NaN,usa


In [14]:
users = users.drop('Location', axis = 1)
users.columns = ['user_id', 'age', 'country']

In [15]:
users.isna().sum()

user_id         0
age        110762
country         0
dtype: int64

In [16]:
users.dtypes

user_id      int64
age        float64
country     object
dtype: object

In [17]:
def age_to_gen(age):
  if age == 0:
    age = 'unknown'
  elif age < 10:
    age = 'under 10'
  elif age < 20:
    age = '10s'
  elif age < 30:
    age = '20s'
  elif age < 40:
    age = '30s'
  elif age < 50:
    age = '40s'
  elif age < 60:
    age = '50s'
  else:
    age = 'over 60'
  
  return age

In [18]:
users.age.fillna(0, inplace = True)
users['gen'] = users['age'].apply(lambda x : age_to_gen(x))

In [19]:
users

,user_id,age,country,gen
0,1,0.0,usa,unknown
1,2,18.0,usa,10s
2,3,0.0,russia,unknown
3,4,17.0,portugal,10s
4,5,0.0,united kingdom,unknown
...,...,...,...,...
278853,278854,0.0,usa,unknown
278854,278855,50.0,united kingdom,50s
278855,278856,0.0,canada,unknown
278856,278857,0.0,usa,unknown


In [20]:
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [22]:
ratings.columns = ['user_id', 'ISBN', 'score']

In [ ]:
rate_table = pd.pivot_table(ratings, index = ['user_id'], columns = ['ISBN'],
                            values = ['score'])